In [ ]:
import torch
print(torch.cuda.is_available())
torch.set_num_threads(8)
import scvi
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import anndata
from time import time
import pyreadr
import json
import psutil

out_path = './manuscript/results/public13Dataset_CITEseq/CSV/'

# ## 13 dataset with 413 genes
run_name = 'public13Dataset_CITEseq'
adts = pyreadr.read_r('./results/publicData_CITEseq/RDS/adt_data_common_RawCount_publicData_CITEseq_12buusT.rds')[None].astype(int)
gex = pyreadr.read_r('./results/publicData_CITEseq/RDS/rna_data_common_RawCount_public13Dataset_CITEseq.rds')[None].astype(int)
obs = pyreadr.read_r('./results/publicData_CITEseq/RDS/adt_feature_publicData_CITEseq_12buusT.rds')[None]


# adts.to_csv('./results/publicData_CITEseq/CSV/adt_data_common_RawCount_'+ run_name + '.csv')
# gex.to_csv('./results/publicData_CITEseq/CSV/rna_data_common_RawCount_'+ run_name + '.csv')
# obs.to_csv('./results/publicData_CITEseq/CSV/adt_feature_common_RawCount_'+ run_name + '.csv')



## T cell dataset
run_name = 'public13Dataset_CITEseq_Tcelllargeprop'
out_path = './manuscript/results/' + run_name + '/CSV/'

adts = pyreadr.read_r('./manuscript/results/' + run_name + '/RDS/adt_data_RawCount_' + run_name + '.rds')[None].astype(int)
gex = pyreadr.read_r('./manuscript/results/' + run_name + '/RDS/rna_data_RawCount_' + run_name + '.rds')[None].astype(int)
obs = pyreadr.read_r('./manuscript/results/' + run_name + '/RDS/adt_feature_' + run_name + '.rds')[None]

adts.to_parquet('./manuscript/results/' + run_name + '/CSV/adt_data_common_RawCount_'+ run_name + '.parquet')
gex.to_parquet('./manuscript/results/' + run_name + '/CSV/rna_data_common_RawCount_'+ run_name + '.parquet')
obs.to_parquet('./manuscript/results/' + run_name + '/CSV/adt_feature_common_RawCount_'+ run_name + '.parquet')

## T cell dataset - CD8
run_name = 'public13Dataset_CITEseq_Tcelllargeprop_cd8T'
out_path = './manuscript/results/' + run_name + '/CSV/'

adts = pyreadr.read_r('./manuscript/results/' + run_name + '/RDS/adt_data_RawCount_' + run_name + '.rds')[None].astype(int)
gex = pyreadr.read_r('./manuscript/results/' + run_name + '/RDS/rna_data_RawCount_' + run_name + '.rds')[None].astype(int)
obs = pyreadr.read_r('./manuscript/results/' + run_name + '/RDS/adt_feature_' + run_name + '.rds')[None]

adts.to_parquet('./manuscript/results/' + run_name + '/CSV/adt_data_common_RawCount_'+ run_name + '.parquet')
gex.to_parquet('./manuscript/results/' + run_name + '/CSV/rna_data_common_RawCount_'+ run_name + '.parquet')
obs.to_parquet('./manuscript/results/' + run_name + '/CSV/adt_feature_common_RawCount_'+ run_name + '.parquet')


adata = anndata.AnnData(gex,obs)
adts.index = adata.obs_names
adata.obsm['protein_expression'] = adts


In [2]:
print(scvi.__version__)

0.6.7


In [ ]:
def log_memory_usage(file):
    memory = psutil.virtual_memory()
    file.write(f'Total memory: {memory.total / (1024.0 ** 3)} GB\n')
    file.write(f'Used memory: {memory.used / (1024.0 ** 3)} GB\n')
    file.write(f'Available memory: {memory.available / (1024.0 ** 3)} GB\n')

In [1]:
# adata.obs['batch_sample'] = adata.obs.study_name.astype(str) + adata.obs['sample'].astype(str)
batch_keys = ['study_name', 'sample']

True


In [ ]:
times = dict()
times['train'] = dict()
times['extract'] = dict()
b = 'study_name'

sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key=b, span =1, subset=True)


with open(out_path + run_name + '_memory_usage.log', 'w') as f:
    log_memory_usage(f)
    start = time()
    scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')
    log_memory_usage(f)
    vae = scvi.model.TOTALVI(adata, latent_distribution="normal", protein_dispersion='protein-batch')
    log_memory_usage(f)
    vae.train(use_gpu = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    train_time = round(end - start,0)
    times['train'][b] = train_time
    log_memory_usage(f)
    start = time()
    rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique(), gene_list=['CD3E'])
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    extract_time = round(end - start,0)
    times['extract'][b] = extract_time
    log_memory_usage(f)

rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{b}_GPU_{run_name}.csv', index = False)
protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{b}_GPU_{run_name}.csv', index = False)
adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{b}_GPU_{run_name}.csv', index = False)

with open(out_path + run_name + '_totalVI_runtime.csv', 'w') as f:
    json.dump(times, f)



In [ ]:
times = dict()
times['train'] = dict()
times['extract'] = dict()
b = 'sample'
# for b in batch_keys:

sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key='study_name', span =1, subset=True)


with open(out_path + run_name + '_memory_usage_' + b + '.log', 'w') as f:
    log_memory_usage(f)
    start = time()
    scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')
    log_memory_usage(f)
    vae = scvi.model.TOTALVI(adata, latent_distribution="normal", protein_dispersion='protein-batch')
    log_memory_usage(f)
    vae.train(use_gpu = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    train_time = round(end - start,0)
    times['train'][b] = train_time
    log_memory_usage(f)
    start = time()
    rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique(), gene_list=['CD3E'])
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    extract_time = round(end - start,0)
    times['extract'][b] = extract_time
    log_memory_usage(f)

rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{b}_GPU_{run_name}.csv', index = False)
protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{b}_GPU_{run_name}.csv', index = False)
adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{b}_GPU_{run_name}.csv', index = False)


with open(out_path + run_name + '_totalVI_runtime_' + b + '.csv', 'w') as f:
    json.dump(times, f)

    # del vae
    # del rna
    # del protein

In [ ]:
times = dict()
times['train'] = dict()
times['extract'] = dict()
b = 'study_name'

sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key=b, span =1, subset=True)


with open(out_path + run_name + '_totalVI_memory_usage_' + b + '_GPU.log', 'w') as f:
    log_memory_usage(f)
    start = time()
    scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')
    log_memory_usage(f)
    vae = scvi.model.TOTALVI(adata, latent_distribution="normal", protein_dispersion='protein-batch')
    log_memory_usage(f)
    vae.train(use_gpu = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    train_time = round(end - start,0)
    times['train'][b] = train_time
    log_memory_usage(f)
    start = time()
    rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique(), gene_list=['CD3E'], include_protein_background = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    extract_time = round(end - start,0)
    times['extract'][b] = extract_time
    log_memory_usage(f)

rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{run_name}_{b}_include_protein_background.csv', index = False)
protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{run_name}_{b}_include_protein_background.csv', index = False)
adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{run_name}_{b}_include_protein_background.csv', index = False)

with open(out_path + run_name + '_totalVI_runtime_' + b + '_GPU.csv', 'w') as f:
    json.dump(times, f)



In [ ]:
times = dict()
times['train'] = dict()
times['extract'] = dict()
b = 'sample'

sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key=b, span =1, subset=True)


with open(out_path + run_name + '_totalVI_memory_usage_' + b + '_GPU.log', 'w') as f:
    log_memory_usage(f)
    start = time()
    scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')
    log_memory_usage(f)
    vae = scvi.model.TOTALVI(adata, latent_distribution="normal", protein_dispersion='protein-batch')
    log_memory_usage(f)
    vae.train(use_gpu = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    train_time = round(end - start,0)
    times['train'][b] = train_time
    log_memory_usage(f)
    start = time()
    rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique(), gene_list=['CD3E'], include_protein_background = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    extract_time = round(end - start,0)
    times['extract'][b] = extract_time
    log_memory_usage(f)

rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{run_name}_{b}_include_protein_background.csv', index = False)
protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{run_name}_{b}_include_protein_background.csv', index = False)
adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{run_name}_{b}_include_protein_background.csv', index = False)

with open(out_path + run_name + '_totalVI_runtime_' + b + '_GPU.csv', 'w') as f:
    json.dump(times, f)



In [ ]:
times = dict()
times['train'] = dict()
times['extract'] = dict()
b = 'study_name'

sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key=b, span =1, subset=True)


with open(out_path + run_name + '_totalVI_memory_usage_' + b + '_GPU_test_withoutcelltypecolumns.log', 'w') as f:
    log_memory_usage(f)
    start = time()
    scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')
    log_memory_usage(f)
    vae = scvi.model.TOTALVI(adata, latent_distribution="normal", protein_dispersion='protein-batch')
    log_memory_usage(f)
    vae.train(use_gpu = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    train_time = round(end - start,0)
    times['train'][b] = train_time
    log_memory_usage(f)
    start = time()
    rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique(), gene_list=['CD3E'], include_protein_background = False)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    extract_time = round(end - start,0)
    times['extract'][b] = extract_time
    log_memory_usage(f)

rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{run_name}_{b}_test_withoutcelltypecolumns.csv', index = False)
protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{run_name}_{b}_test_withoutcelltypecolumns.csv', index = False)
adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{run_name}_{b}_test_withoutcelltypecolumns.csv', index = False)

with open(out_path + run_name + '_totalVI_runtime_' + b + '_GPU_test_withoutcelltypecolumns.csv', 'w') as f:
    json.dump(times, f)



In [ ]:
times = dict()
times['train'] = dict()
times['extract'] = dict()
b = 'sample'

sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key=b, span =1, subset=True)


with open(out_path + run_name + '_totalVI_memory_usage_' + b + '_GPU_test_withoutcelltypecolumns.log', 'w') as f:
    log_memory_usage(f)
    start = time()
    scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')
    log_memory_usage(f)
    vae = scvi.model.TOTALVI(adata, latent_distribution="normal", protein_dispersion='protein-batch')
    log_memory_usage(f)
    vae.train(use_gpu = True)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    train_time = round(end - start,0)
    times['train'][b] = train_time
    log_memory_usage(f)
    start = time()
    rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique(), gene_list=['CD3E'], include_protein_background = False)
    log_memory_usage(f)
    end = time()
    print(f'Seconds to complete: {round(end - start,0)}')
    extract_time = round(end - start,0)
    times['extract'][b] = extract_time
    log_memory_usage(f)

rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{run_name}_{b}_test_withoutcelltypecolumns.csv', index = False)
protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{run_name}_{b}_test_withoutcelltypecolumns.csv', index = False)
adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{run_name}_{b}_test_withoutcelltypecolumns.csv', index = False)

with open(out_path + run_name + '_totalVI_runtime_' + b + '_GPU_test_withoutcelltypecolumns.csv', 'w') as f:
    json.dump(times, f)



In [ ]:
## write out the totalVI integration embedding and gene normalized values

b = 'study_name'
sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key=b, span =1, subset=True)

scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')

vae = scvi.model.TOTALVI(adata, latent_distribution="normal", protein_dispersion='protein-batch')

vae.train(use_gpu = True)

rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique())

latent = vae.get_latent_representation()

rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{b}_GPU_{run_name}_fullgenelist.csv', index = False)
# protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{b}_GPU_{run_name}.csv', index = False)
# adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{b}_GPU_{run_name}.csv', index = False)

pd.DataFrame(latent).to_csv(f'{out_path}/latent_totalVI_{b}_GPU_{run_name}.csv', index = False)




In [ ]:
## scvi normalizataion
b = 'study_name'
adata = anndata.AnnData(gex, obs)
adts.index = adata.obs_names

scvi.model.SCVI.setup_anndata(adata, batch_key=b)
vae = scvi.model.SCVI(adata)
vae.train(use_gpu = True)
adata.obsm["X_scVI"] = vae.get_latent_representation()
adata.obsm["X_normalized_scVI"] = vae.get_normalized_expression()

rna = adata.obsm["X_normalized_scVI"]
latent = adata.obsm["X_scVI"]

rna.to_csv(f'{out_path}/cell_x_gene_scVI_{b}_GPU_{run_name}_fullgenelist.csv', index = False)
pd.DataFrame(latent).to_csv(f'{out_path}/cell_x_gene_latent_scVI_{b}_GPU_{run_name}.csv', index = False)

sc.tl.pca(adata, n_comps = 50)
sc.pp.neighbors(adata, n_pcs=50, n_neighbors=20)
sc.tl.umap(adata)


fig_path = "./manuscript/results/public13Dataset_CITEseq/Figures/scVI_umap_studyname.pdf"
sc.pl.umap(
    adata,
    color=["study_name"],
    frameon=False,
    show = False
)
fig_path = "./manuscript/results/public13Dataset_CITEseq/Figures/scVI_umap_studyname.pdf"
plt.savefig(fig_path)
plt.close()

sc.pl.umap(
    adata,
    color=["cell_type_l1"],
    ncols=2,
    frameon=False,
    show = False
)
fig_path = "./manuscript/results/public13Dataset_CITEseq/Figures/scVI_umap_cell_type_l1.pdf"
plt.savefig(fig_path)
plt.close()

sc.write('./manuscript/results/public13Dataset_CITEseq/H5ad/rna_scVI_default_adata_pc50.h5ad', adata)
# adata_loaded = sc.read('/path/to/your/directory/adata.h5ad')


In [ ]:
## try using ADTnorm results as the protein expression values
run_name = 'public13Dataset_CITEseq'
adts = pyreadr.read_r('./manuscript/results/public13Dataset_CITEseq/RDS/adt_data_norm_ADTnorm_study_manual_keepZero_public13Dataset_CITEseq.rds')[None].astype(int)
gex = pyreadr.read_r('./results/publicData_CITEseq/RDS/rna_data_common_RawCount_public13Dataset_CITEseq.rds')[None].astype(int)
obs = pyreadr.read_r('./results/publicData_CITEseq/RDS/adt_feature_publicData_CITEseq_12buusT.rds')[None]
## write out the totalVI integration embedding and gene normalized values
adata = anndata.AnnData(gex,obs)
adts.index = adata.obs_names
adata.obsm['protein_expression'] = adts

b = 'study_name'
sc.pp.highly_variable_genes(adata, n_top_genes=4000, flavor='seurat_v3', batch_key=b, span =1, subset=True)

scvi.model.TOTALVI.setup_anndata(adata, batch_key = b, protein_expression_obsm_key = 'protein_expression')

vae = scvi.model.TOTALVI(adata, n_latent = 20, latent_distribution="normal", protein_dispersion='protein-batch')
# vae = scvi.model.TOTALVI(adata, n_latent = 20, latent_distribution="normal", protein_dispersion='protein')

vae.train(use_gpu = True)

# rna, protein = vae.get_normalized_expression(n_samples = 25, return_mean = True, transform_batch = adata.obs[b].unique())

latent = vae.get_latent_representation()

# rna.to_csv(f'{out_path}/cell_x_gene_totalVI_{b}_GPU_{run_name}_fullgenelist.csv', index = False)
# protein.to_csv(f'{out_path}/cell_x_adt_totalVI_{b}_GPU_{run_name}.csv', index = False)
# adata.obs.to_csv(f'{out_path}/cell_x_feature_totalVI_{b}_GPU_{run_name}.csv', index = False)

pd.DataFrame(latent).to_csv(f'{out_path}/latent_totalVI_onADTnorm_manual_study_{b}_GPU_{run_name}_latent20.csv', index = False)
# pd.DataFrame(latent).to_csv(f'{out_path}/latent_totalVI_onADTnorm_manual_study_{b}_GPU_{run_name}_latent20_protein_dispersion_protein.csv', index = False)

